# Here is a title
This version open each folder (balance training, unbalance training and eval) separately

In [2]:
#%%capture 
#hide output of this cell
import os
import tensorflow as tf
print(tf.__version__)


# Qua fatti una seconda var con il tuo path, cosi a seconda di chi lo sta usando mettiamo un hastag allinizio
tfrecord_path = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/bal_train/--.tfrecord'
# tfrecord_path = '/Users/giacomo/Documents/lavoro/thunder_recognition/audioset_v1_embeddings/bal_train/--.tfrecord'

2.14.0


First of all, inspect the raw TFR file structure to get an idea of how its built and it looks. 

In [3]:
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Iterate through the dataset and print the raw example data
n = 1
for record in dataset:
    #print('record nr ', n)
    example = tf.train.SequenceExample.FromString(record.numpy())
    tf.print(example.feature_lists.feature_list["audio_embedding"])
    n += 1

feature {
  bytes_list {
    value: "_z\223\016e0\235}\372\365j\2513~M\221\205%\265\237 \373\327\210\240\367\232+_\033\210`q=H\377J[\247j\003\230p\377\371a\334\3770\377\177\377*\030\211\251\226\340\000C\370\246:T\000\2177d\253\377|E\177\006\000\000+\000\377\207EO\323\261\321\272\342Mk\225\033\377\036\377\000\034\207\377\374o_\377\233\\C\343Na\000\377\000\000\333q\377\232m\177\271\000\272\000?\377\377\002\276\000"
  }
}
feature {
  bytes_list {
    value: "{m\211!\221~+Z\375\266p~3\202\211T\252\212Gl~~\377u\234o\252y\\\000H\200\310\000\250~}\214\231q\210#\000f\270\306m\315\374\212\t\377C=+\2747\317#\271\365(\000\371;H\005F\202\276\231\251\241\377r\240\350\220\227p\235\000\315qp\377\017nJ\241\000\244\224\306\313~\264\231\2723\367\271\310\377\000x\305\377</\207)n\226\232Cp\2468\235\\\213\000\032T{\214\370"
  }
}
feature {
  bytes_list {
    value: "4\207\230\007>pyS\374\301!W5\255|w^\0149iF\257\360\220}\237\334\033U\000p\321\355\000\231\1779U\204\024Li\001\377\377\320\340\377\217\377\000\

Now iterate trough all files of the google AudioSet dataset. Using `filter_for_thunder()` we isolate all entries that have the thunder label and create a new file with them. 

In [4]:
# minimal example, file loop:

#tfrecord_folder = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/bal_train'
tfrecord_folder = '/Users/giacomo/Documents/lavoro/thunder_recognition/audioset_v1_embeddings/unbal_train/'

tfrecord_files = [os.path.join(tfrecord_folder, f) for f in os.listdir(tfrecord_folder) if f.endswith('.tfrecord')]
print(len(tfrecord_files))

1444


In [5]:
# Create a TFRecord writer for the new file
def filter_for_thunder(tfrecord_folder, new_file_path, value_to_check):
    """Parse trough a tfrecord file and search for entries related to one specific label, for example thunder (287),
    when found, append the whole entrie to a new file.

    Args:
        old_file_path (str): path of the original TFRecord file
        new_file_path (str): path to the newly created file containing only the selected entries
        value_to_check (int): value of the label to use as filter 
        entries_list (empty list): an empty list to append matching entries to 
    """   
    
    #create a list with all tfrecord files 
    tfrecord_files = [os.path.join(tfrecord_folder, f) for f in os.listdir(tfrecord_folder) if f.endswith('.tfrecord')]
    entries_list = [] #empty list to append matching records
    label_counter = 0 #counter to count the number of matching labels
    all_counter = 0
    
    for file in tfrecord_files:
          
        for raw_record in tf.data.TFRecordDataset(file):
            example = tf.train.SequenceExample.FromString(raw_record.numpy())
            #example.features.feature['labels'].int64_list.value
            #print(example.features.feature['labels'].int64_list.value, 'all')
            all_counter += 1

            if value_to_check in example.context.feature['labels'].int64_list.value:
                # If the value is found, write the example to the new TFRecord file
                entries_list.append(raw_record.numpy())                        #not sure if the .numpy() is needed here, in case of problems delete!
                label_counter += 1
                #print(label_counter, raw_record.numpy())
                print(example.context.feature['labels'].int64_list.value, 'picked')
                print(example.feature_lists.feature_list["audio_embedding"])
            else:
                print(example.context.feature['labels'].int64_list.value, 'not picked')
                
    
    print('I found', label_counter, ' records for label', value_to_check, 'on ', all_counter, ' total record')                    
                        
    with tf.io.TFRecordWriter(new_file_path) as writer:
        for entry in entries_list:
            writer.write(entry)                    
                        
    return entries_list

In [6]:
# use filter_for_thunder on the bal_training files 
tfrecord_folder = '/Users/giacomo/Documents/lavoro/thunder_recognition/audioset_v1_embeddings/bal_train/'
new_filename = 'thunder_bal_training.tfrecord'

entries_list = filter_for_thunder(tfrecord_folder, new_filename, value_to_check=287)

[293] not picked
[521] not picked
[137, 489] not picked
[33, 137] not picked
[0, 417] not picked
[0, 435] not picked
[137, 138, 139, 140, 189, 463, 511, 517] not picked
[137, 234] not picked
[348, 488] not picked
[34, 137] not picked
[65] not picked
[96, 97] not picked
[32, 137] not picked
[243] not picked
[0, 72, 73, 81, 85] not picked
[126, 128, 129] not picked
[427, 429] not picked
[178] not picked
[0, 400] not picked
[288, 370, 371] not picked
[137, 278] not picked
[436] not picked
[114] not picked
[0, 345, 425] not picked
[98, 102, 103] not picked
[137, 151] not picked
[0, 7] not picked
[228] not picked
[135, 137] not picked
[185, 187] not picked
[457] not picked
[155, 157] not picked
[0, 382, 506] not picked
[293] not picked
[0, 355] not picked
[126, 129, 131] not picked
[0, 447, 448] not picked
[0, 18, 137] not picked
[374, 451] not picked
[373] not picked
[137, 138, 209, 230] not picked
[137, 248, 249] not picked
[25] not picked
[54] not picked
[347] not picked
[130] not picked

In [7]:
# use filter_for_thunder on the eval files 
#tfrecord_folder = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/eval'
tfrecord_folder = '/Users/giacomo/Documents/lavoro/thunder_recognition/audioset_v1_embeddings/eval/'
new_filename = 'thunder_eval.tfrecord'

filter_for_thunder(tfrecord_folder, new_filename, value_to_check=287)



[0, 393] not picked
[0, 137, 507, 508] not picked
[300, 307, 348, 353] not picked
[137, 471] not picked
[0, 3, 446, 510] not picked
[0, 137, 138, 164, 169] not picked
[47] not picked
[137, 147, 229, 231] not picked
[137, 275] not picked
[418, 419] not picked
[137, 269] not picked
[177] not picked
[0, 22, 47] not picked
[137, 401] not picked
[459, 504] not picked
[461] not picked
[0, 470] not picked
[387] not picked
[0, 22] not picked
[314] not picked
[0, 300, 307, 351, 352] not picked
[0, 49] not picked
[137, 218] not picked
[453] not picked
[300, 335, 338] not picked
[0, 137, 500, 506] not picked
[367, 455, 490] not picked
[0, 55, 506] not picked
[179, 180, 181] not picked
[27, 137, 224, 230, 249, 263, 266] not picked
[27, 137, 256, 258, 497] not picked
[137, 220, 281] not picked
[137, 253, 278] not picked
[98, 99, 100] not picked
[185, 186] not picked
[0, 137, 394] not picked
[230] not picked
[137, 249] not picked
[390] not picked
[171, 172] not picked
[137, 300, 320] not picked
[185

[b'\nq\n\x1e\n\x12start_time_seconds\x12\x08\x12\x06\n\x04\x00\x00\xc8B\n\x14\n\x06labels\x12\n\x1a\x08\n\x06\x9e\x02\x9f\x02\xa1\x02\n\x1c\n\x10end_time_seconds\x12\x08\x12\x06\n\x04\x00\x00\xdcB\n\x1b\n\x08video_id\x12\x0f\n\r\n\x0b_Vgj16TXFdU\x12\xf1\n\n\xee\n\n\x0faudio_embedding\x12\xda\n\n\x86\x01\n\x83\x01\n\x80\x01\x1d\xc9L\x00\xc4f\xe3\xc3\xa6\xae\xa3\xff\xff\xcd]\xd9\xc5\xa3%F\x94\xff\xff\xc0\xa1\x04\xae\xb6\x1a\xa5\x00\x87\xaaM\xff\xfa;\x00\xff\xff\xc2Z\xff\xff\x00\xa4\xe3X\x00\x1ef\xa1{\xffe\xff\xa9\xff\xffG\xff\xff\xff\xb2\x1d\x00\xfffc\xa4W\x00\xb30\x98cW\xa5)\xf6\x87T{\x96\x00i\x00\x82\x84\x00C\x00\xb05\x9c\x00\xd1\xd7\x89\xa7o\xff\xffd\x00\xccGuF\xff\x00\xbf$\x91.m\x00\xff\xc7\x00\xcd\xc4\xa2e\xff\x86k\x00\n\x86\x01\n\x83\x01\n\x80\x01C\x95\x80\x00\xea5\xa8\xaa\xa7\x95\xc8\xff\x84\xb1\xe9\xd08~\x83\x00x\xff\xae\x924\x95\xb5@R\x9c\x83\x88\xde\x91\xcf\xb20\x11\xa6\xff\x8ek\xacxk\x8ftx\x00v}&b\xff\x00\xffSG\xe4\xa4\xffQt\x1b;V\xffs\x92\xd4\x9a\x99Rs\x00\x87\xbc\xa7\x80\xd5

In [8]:
# use filter_for_thunder on the unbalanced training files 

new_filename = 'thunder_bal_train.tfrecord'

filter_for_thunder(tfrecord_folder, new_filename, value_to_check=287)



[0, 393] not picked
[0, 137, 507, 508] not picked
[300, 307, 348, 353] not picked
[137, 471] not picked
[0, 3, 446, 510] not picked
[0, 137, 138, 164, 169] not picked
[47] not picked
[137, 147, 229, 231] not picked
[137, 275] not picked
[418, 419] not picked
[137, 269] not picked
[177] not picked
[0, 22, 47] not picked
[137, 401] not picked
[459, 504] not picked
[461] not picked
[0, 470] not picked
[387] not picked
[0, 22] not picked
[314] not picked
[0, 300, 307, 351, 352] not picked
[0, 49] not picked
[137, 218] not picked
[453] not picked
[300, 335, 338] not picked
[0, 137, 500, 506] not picked
[367, 455, 490] not picked
[0, 55, 506] not picked
[179, 180, 181] not picked
[27, 137, 224, 230, 249, 263, 266] not picked
[27, 137, 256, 258, 497] not picked
[137, 220, 281] not picked
[137, 253, 278] not picked
[98, 99, 100] not picked
[185, 186] not picked
[0, 137, 394] not picked
[230] not picked
[137, 249] not picked
[390] not picked
[171, 172] not picked
[137, 300, 320] not picked
[185

[b'\nq\n\x1e\n\x12start_time_seconds\x12\x08\x12\x06\n\x04\x00\x00\xc8B\n\x14\n\x06labels\x12\n\x1a\x08\n\x06\x9e\x02\x9f\x02\xa1\x02\n\x1c\n\x10end_time_seconds\x12\x08\x12\x06\n\x04\x00\x00\xdcB\n\x1b\n\x08video_id\x12\x0f\n\r\n\x0b_Vgj16TXFdU\x12\xf1\n\n\xee\n\n\x0faudio_embedding\x12\xda\n\n\x86\x01\n\x83\x01\n\x80\x01\x1d\xc9L\x00\xc4f\xe3\xc3\xa6\xae\xa3\xff\xff\xcd]\xd9\xc5\xa3%F\x94\xff\xff\xc0\xa1\x04\xae\xb6\x1a\xa5\x00\x87\xaaM\xff\xfa;\x00\xff\xff\xc2Z\xff\xff\x00\xa4\xe3X\x00\x1ef\xa1{\xffe\xff\xa9\xff\xffG\xff\xff\xff\xb2\x1d\x00\xfffc\xa4W\x00\xb30\x98cW\xa5)\xf6\x87T{\x96\x00i\x00\x82\x84\x00C\x00\xb05\x9c\x00\xd1\xd7\x89\xa7o\xff\xffd\x00\xccGuF\xff\x00\xbf$\x91.m\x00\xff\xc7\x00\xcd\xc4\xa2e\xff\x86k\x00\n\x86\x01\n\x83\x01\n\x80\x01C\x95\x80\x00\xea5\xa8\xaa\xa7\x95\xc8\xff\x84\xb1\xe9\xd08~\x83\x00x\xff\xae\x924\x95\xb5@R\x9c\x83\x88\xde\x91\xcf\xb20\x11\xa6\xff\x8ek\xacxk\x8ftx\x00v}&b\xff\x00\xffSG\xe4\xa4\xffQt\x1b;V\xffs\x92\xd4\x9a\x99Rs\x00\x87\xbc\xa7\x80\xd5

In [9]:
# check the newly created file if it containes the entries we expect
dataset2 = tf.data.TFRecordDataset(new_filename)

# Iterate through the dataset and print the raw example data
for record in dataset2:
    example = tf.train.SequenceExample.FromString(record.numpy())
    #print(example.feature_lists.feature_list["audio_embedding"].feature[0].bytes_list.value)
    #print(len(example.feature_lists.feature_list["audio_embedding"].feature))

In [12]:

import soundfile as sf  # To save audio as WAV or MP3 (you may need to install this package)

# Define a function to convert audio embeddings to audio signal
# You may need to adjust these parameters according to your datase
def embeddings_to_audio(embeddings):
    num_channels = 1  # Mono audio, change to 2 for stereo audio
    sample_rate = 128

    # Decode the embeddings (assuming they are float32)
    audio_signal = tf.io.decode_raw(embeddings, out_type=tf.float32)
    
    # Reshape the audio signal to the correct shape
    audio_signal = tf.reshape(audio_signal, shape=[-1, num_channels])

    return audio_signal

# Path to the TFRecord file
new_filename = "thunder_bal_training.tfrecord"
dataset = tf.data.TFRecordDataset(new_filename)

# Iterate through the dataset and save audio as WAV or MP3
for record in dataset:
    example = tf.train.SequenceExample.FromString(record.numpy())
    audio_embeddings = example.feature_lists.feature_list["audio_embedding"].feature[0].float_list.value
    
    # Convert audio embeddings to audio signal
    audio_signal = embeddings_to_audio(audio_embeddings)
    
    # Save the audio as a WAV or MP3 file
    audio_signal_np = audio_signal.numpy()
    save_path = "output_audio.wav"  # Change the file extension to .mp3 for MP3 format
    sf.write(save_path, audio_signal_np, 128)

In [51]:
import soundfile as sf

# Define a function to convert audio embeddings to audio signal
def embeddings_to_audio(embeddings):
    sample_rate = 16000  # Adjust to your dataset's sample rate
    num_channels = 1  # Adjust for mono/stereo audio

    audio_signal = tf.io.decode_raw(embeddings, out_type=tf.float32)
    audio_signal = tf.reshape(audio_signal, shape=[-1, num_channels])

    return audio_signal

# Path to the TFRecord file
new_filename = "thunder_bal_training.tfrecord"
dataset = tf.data.TFRecordDataset(new_filename)

# Define a function to process each record
def process_record(record):
    example = tf.train.SequenceExample.FromString(record)
    audio_embeddings = example.feature_lists.feature_list["audio_embedding"].feature[0].bytes_list.value
    audio_signal = embeddings_to_audio(audio_embeddings)
    return audio_signal

# Use the map function to process the dataset
processed_dataset = dataset2.map(process_record)

# Define a function to save audio as WAV or MP3
def save_audio(audio_signal, save_path):
    with tf.compat.v1.Session() as sess:
        audio_signal_np = tf.make_ndarray(audio_signal)
        sf.write(save_path, audio_signal_np, sample_rate)

# Iterate over the processed dataset and save audio
for audio_signal in processed_dataset:
    save_path = "output_audio.wav"  # Change the file extension to .mp3 for MP3 format
    save_audio(audio_signal, save_path)

TypeError: in user code:

    File "/var/folders/65/jjhht_kn6479d_8drvxjcgjh0000gn/T/ipykernel_63950/1665377163.py", line 19, in process_record  *
        example = tf.train.SequenceExample.FromString(record)

    TypeError: expected bytes, SymbolicTensor found


# Junk code, keep for now, delete later

In [ ]:
# Nuovo approcio da stack overflow: https://stackoverflow.com/questions/65007191/how-to-read-decode-tfrecords-file-see-the-images-inside-and-do-augmentation
# questo funyiona un pó meglio, si riescono ad accedere le singole entry. Sembra che alla fine file 
# TFRecord siano dizionari di dizionari di dizionari o qualcosa del genere. 
dataset = tf.data.TFRecordDataset(tfrecord_path)


val = 1
info = {}


for i, raw_record in enumerate(dataset): # keeping at a minimal example of 3, remove for actual code
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    
    # Questa parte qua fa il check per vedere se trova entries con thunder (287)
    if 'labels' in example.features.feature:
        label_values = example.features.feature['labels'].int64_list.value
        if 399 in label_values:
            print('THUNDEEEEERRRRR')
            print(example)
    
    
"""     for k, v in example.features.feature.items():
        if k == 'labels':
            #print(k, v, 'NEXT')
            info[k] = v
            
            val += 1 """
            
            
            

""" print('INFO: \n', info) """
#for n, m in info.items():
 #   if 287 in m:
  #      print('THUNDEERRRRRR')




            

In [ ]:
# Qua faccio un check per capire il data type del feature labels, perché piú avanti mi crea problemi
# Define a parsing function to extract and decode the data
## Boh sto qua semplicemente restituisce il data type specificato su tf.io.VarLenFeature(tf.int64), 
## e non il data type reale. Anche cambiare la parte 'features/feature/video_id' non ha nessun effetto


def parse_function(example_proto):
    feature_description = {
        'features/feature/video_id': tf.io.VarLenFeature(tf.int64),
        # Add other features and their types based on your TFRecord structure
    }

    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Get the 'labels' feature
    labels = example['features/feature/video_id']
    
    return labels

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset
parsed_data = dataset.map(parse_function)

# Print the data type of the 'labels' feature
for labels in parsed_data:
    dtype = labels.values.dtype
    print(f'Data Type of "labels" Feature: {dtype}')

Open and explore TFRecord files. 
Goal is to filter them in order to only keep files containing a certain label, like thunder

In [ ]:
#Questo approcio é quello che mi da chatGTP, che peró per me non funziona, le entry che mi ritorna sono vuote. 
# Qua poi in teoria volgio leggere i label (e solo quelli) in modo che piú avanti si possano filtrare i file che contengono il label thunder (287,/m/0ngt1)
# La lista dei vari label si puó trovare nel file csv che ho caricato

# Define a parsing function to extract and decode the data
def parse_function(example_proto):
    feature_description = {
        'features/feature/': tf.io.VarLenFeature(tf.float32),
        # Add other features and their types based on your TFRecord structure
    }

    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode the 'labels' feature
    labels = tf.sparse.to_dense(example['features/feature/'])
    
    return labels

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset and iterate through the records
for labels in dataset.map(parse_function):
    # Print the 'labels' feature
    print(f'time: {labels.numpy()}')

In [ ]:
tfrecord_path = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/bal_train/-0.tfrecord'

# tfrecord_dataset = tf.data.TFRecordDataset(tfrecord_path)


# Define a parsing function to extract and decode the data
def parse_function(example_proto):
    feature_description = {
        'context/feature/video_id': tf.io.FixedLenFeature([], tf.string),
        'context/feature/start_time_seconds': tf.io.FixedLenFeature([], tf.float32),
        'context/feature/end_time_seconds': tf.io.FixedLenFeature([], tf.float32),
        'context/feature/labels': tf.io.VarLenFeature(tf.int64),
        'feature_lists/feature_list/audio_embedding': tf.io.VarLenFeature(tf.string),
        # Define other features and their types based on your TFRecord structure
    }
    
    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # If 'end_time_seconds' may not be present in all records, use allow_missing=True
    if 'context/feature/end_time_seconds' in example:
        end_time_seconds = example['context/feature/end_time_seconds']
    else:
        end_time_seconds = 0.0  # Provide a default value
    
    
    # Decode the VarLenFeatures
    example['context/feature/labels'] = tf.sparse.to_dense(example['context/feature/labels'], default_value=0)
    example['feature_lists/feature_list/audio_embedding'] = tf.sparse.to_dense(example['feature_lists/feature_list/audio_embedding'], default_value='b')
    
    return example

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset and iterate through the records
for record in dataset.map(parse_function):
    video_id = tf.strings.decode(record['context/feature/video_id'], 'utf-8')
    start_time_seconds = tf.make_ndarray(record['context/feature/start_time_seconds'])[0]
    end_time_seconds = tf.make_ndarray(record['context/feature/end_time_seconds'])[0]
    labels = tf.make_ndarray(record['context/feature/labels'])
    audio_embedding = record['feature_lists/feature_list/audio_embedding']
    
    print(f'Video ID: {video_id}')
    print(f'Start Time (seconds): {start_time_seconds}')
    print(f'End Time (seconds): {end_time_seconds}')
    print(f'Labels: {labels}')
    print(f'Audio Embedding: {audio_embedding}')

In [ ]:
# 